In [53]:
import numpy as np
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from yhelper import neural_modeling, overall_cleaner
import pickle
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPool1D, Dropout, Bidirectional, Conv1D, Activation

# Binned (Good vs Bad)

In [55]:
df = pickle.load(open('post_eda/eda_az.p', 'rb'))

In [56]:
text_df = overall_cleaner(df, ['text', 'review_rating'])

In [57]:
text_df.review_rating.replace(1.0, False, inplace=True)
text_df.review_rating.replace(2.0, False, inplace=True)
text_df.review_rating.replace(3.0, False, inplace=True)
text_df.review_rating.replace(4.0, True, inplace=True)
text_df.review_rating.replace(5.0, True, inplace=True)

In [58]:
text_df

,text,review_rating
5201838,i find this to be one of the better walmarts i...,True
678826,best place ever very authentic staff super nic...,True
2683530,the bean and cheese burrito with green sauce i...,True
5802832,i guess to ups oclock means pm i tried to ma...,False
2197113,i am the vice president of product marketing f...,True
...,...,...
3750456,always looking for a great cup with knowledgea...,False
2781437,stopped in after mtb south mountain my boyfrie...,True
302891,the food here is some of the best ive had in a...,True
3312023,loved the arizona roll but the spicy tuna roll...,False


In [59]:
text_srs = text_df.text
total_vocab = set(word for sentence in text_srs for word in sentence.split(' '))

In [60]:
tokenizer = text.Tokenizer(num_words=len(total_vocab))
tokenizer.fit_on_texts(text_srs)

In [61]:
tokenized_list = tokenizer.texts_to_sequences(text_srs)

In [62]:
padded_seq = sequence.pad_sequences(tokenized_list, maxlen=150)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(padded_seq, pd.get_dummies(text_df.review_rating), test_size=0.2)

In [64]:
padded_seq

array([[    0,     0,     0, ...,  1366,     4,    28],
       [    0,     0,     0, ...,  8565,    16,    30],
       [    0,     0,     0, ...,   344,     2,   510],
       ...,
       [    0,     0,     0, ...,     5,    89,    30],
       [    0,     0,     0, ...,   133,    41,   495],
       [    0,     0,     0, ...,    35, 35867,   372]], dtype=int32)

In [65]:
pd.get_dummies(text_df.review_rating).shape

(94946, 2)

In [66]:
model = Sequential()
embedding_size=150

model.add(Embedding(len(total_vocab), embedding_size))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 150)         18442500  
_________________________________________________________________
bidirectional_5 (Bidirection (None, None, 128)         110080    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 258       
Total params: 18,552,838
Trainable params: 18,552,838
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.fit(X_train, y_train, epochs=5, batch_size = 1500, validation_split=0.1)

Train on 68360 samples, validate on 7596 samples
Epoch 1/5
68360/68360 [==============================] - 126s 2ms/step - loss: 0.5867 - acc: 0.6978 - val_loss: 0.4464 - val_acc: 0.7880
Epoch 2/5
68360/68360 [==============================] - 129s 2ms/step - loss: 0.2654 - acc: 0.8921 - val_loss: 0.2271 - val_acc: 0.9081
Epoch 3/5
68360/68360 [==============================] - 127s 2ms/step - loss: 0.1717 - acc: 0.9362 - val_loss: 0.2214 - val_acc: 0.9146
Epoch 4/5
68360/68360 [==============================] - 124s 2ms/step - loss: 0.1299 - acc: 0.9546 - val_loss: 0.2329 - val_acc: 0.9142
Epoch 5/5
68360/68360 [==============================] - 129s 2ms/step - loss: 0.0960 - acc: 0.9683 - val_loss: 0.2688 - val_acc: 0.9107


In [68]:
model.evaluate(X_test, y_test)

18990/18990 [==============================] - 15s 795us/step


[0.26795434442208027, 0.9058978409500986]

In [69]:
model.metrics_names

['loss', 'acc']

# Glove

In [39]:
def create_corpus_new(df):
    corpus=[]
    for review in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(review)]
        corpus.append(words)
    return corpus   

In [40]:
corpus = create_corpus_new(df)

100%|██████████| 94946/94946 [00:32<00:00, 2903.78it/s]


In [41]:
embedding_dict={}
with open('glove.6B.300d.txt','r') as f:
    for line in f:
        values=line.split()
        word = values[0]
        vectors=np.asarray(values[1:],'float64')
        embedding_dict[word]=vectors
f.close()

In [42]:
MAX_LEN=150
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

glove_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [43]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

Number of unique words: 122780


In [44]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,300))

for word,i in tqdm(word_index.items()):
    if i < num_words:
        emb_vec=embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i]=emb_vec           

100%|██████████| 122780/122780 [00:00<00:00, 671788.11it/s]


In [45]:
glove_pad.shape

(94946, 150)

In [46]:
model = Sequential()
embedding_size=150

model.add(Embedding(150000, embedding_size))
model.add(Dropout(0.2))
model.add(Conv1D(128,
                 kernel_size=5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'mse'])

model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 150)         22500000  
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 150)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         96128     
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 256)         263168    
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 256)               0         
___________________________________________________________

In [48]:
X_train,X_test,y_train,y_test=train_test_split(glove_pad, pd.get_dummies(df.review_rating),test_size=0.2)

In [50]:
model.fit(X_train, y_train, epochs=5, batch_size=1500, validation_split=0.1)

Train on 68360 samples, validate on 7596 samples
Epoch 1/5
68360/68360 [==============================] - 310s 5ms/step - loss: 1.2771 - acc: 0.5029 - mean_squared_error: 0.1280 - val_loss: 1.0823 - val_acc: 0.5943 - val_mean_squared_error: 0.1102
Epoch 2/5
68360/68360 [==============================] - 300s 4ms/step - loss: 0.9083 - acc: 0.6289 - mean_squared_error: 0.0959 - val_loss: 0.8094 - val_acc: 0.6643 - val_mean_squared_error: 0.0869
Epoch 3/5
68360/68360 [==============================] - 304s 4ms/step - loss: 0.7131 - acc: 0.7052 - mean_squared_error: 0.0781 - val_loss: 0.7596 - val_acc: 0.6892 - val_mean_squared_error: 0.0825
Epoch 4/5
68360/68360 [==============================] - 298s 4ms/step - loss: 0.6079 - acc: 0.7521 - mean_squared_error: 0.0675 - val_loss: 0.7666 - val_acc: 0.6836 - val_mean_squared_error: 0.0825
Epoch 5/5
68360/68360 [==============================] - 283s 4ms/step - loss: 0.5305 - acc: 0.7906 - mean_squared_error: 0.0590 - val_loss: 0.8130 - val_a

In [51]:
model.evaluate(X_test, y_test)

18990/18990 [==============================] - 22s 1ms/step


[0.8217614086820553, 0.6801474460179457, 0.08627992903969676]

In [52]:
model.metrics_names

['loss', 'acc', 'mean_squared_error']